In [7]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

# Create a console handler and set its level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it on the console handler
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


def main():
    logger.info("Cloud function starting.")
    # 1. Scrape coingecko gainers.
    import scraping.scraper

    scraping.scraper.main()

    # 2. Get contract address for gainers that have them, with coingecko.
    import scraping.contract_addresses

    scraping.contract_addresses.main(local=True)

    # 3. Run SQL query on Google Big Query to find wallets that have receieved these tokens within the last 7 days.
    import gbq.gbq

    gbq.gbq.main(local=True)


main()


2023-04-04 11:25:40,846 - INFO - Cloud function starting.
2023-04-04 11:25:43,083 - INFO - Scraping complete.
2023-04-04 11:25:43,741 - INFO - Current working directory: /Users/dbless/Documents/Github/cryptocurrency-notifications/cloud_functions/coingecko_gainers_traders
2023-04-04 11:25:45,514 - INFO - Scraping 3 coins that are missing symbols.
2023-04-04 11:25:50,639 - INFO - Scraped https://www.coingecko.com/en/coins/crypto-gladiator-league
2023-04-04 11:25:54,330 - INFO - Scraped https://www.coingecko.com/en/coins/hnc-coin
2023-04-04 11:25:58,004 - INFO - Scraped https://www.coingecko.com/en/coins/jenny-dao-v1
2023-04-04 11:25:58,021 - INFO - Number of coins with ethereum contract addresses: 33
2023-04-04 11:25:58,022 - INFO - Done scraping contract addresses!
2023-04-04 11:25:58,027 - INFO - Checking 33 token contracts.
2023-04-04 11:25:58,285 - INFO - Running SQL query on Google Big Cloud.
2023-04-04 11:26:01,379 - INFO - Query will process 764 MB.
2023-04-04 11:26:01,380 - INFO 

In [21]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
import requests
from google.cloud import bigquery
from google.oauth2 import service_account
import google.auth.transport.requests
import google.oauth2.id_token
import json

load_dotenv()
KEY_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
GOOGLE_CLOUD_FUNCTION = os.getenv("GOOGLE_CLOUD_FUNCTION")
endpoint = GOOGLE_CLOUD_FUNCTION
audience = endpoint

auth_req = google.auth.transport.requests.Request()
id_token = google.oauth2.id_token.fetch_id_token(auth_req, audience)

# Add message in request args
headers = {"Authorization": f"Bearer {id_token}", "Content-Type": "application/json"}


r = requests.post(
    endpoint, data=json.dumps({"message": "wow json data"}), headers=headers
)


In [30]:
r.content


b'PK\x03\x04\x14\x00\x00\x00\x00\x00\x0f\xa0\x84V\xfb\xaa}\xaf\xdf\x0e\x00\x00\xdf\x0e\x00\x00\x10\x00\x00\x00/tmp/gainers.csvname,price,change,href_id,api_id,symbol,contract_address\nUwU Lend,$12.82,122.4%,uwu-lend,uwu-lend,uwu,0x55c08ca52497e2f1534b59e2917bf524d4765257\nLinear,$0.016050588560,30.6%,linear,linear,lina,0x3e9bc21c9b189c09df3ef1b824798658d5011937\nStrips Finance,$1.11,28.3%,strips-finance,strips-finance,strp,0x97872eafd79940c7b24f7bcc1eadb1457347adc9\nShib Original Vision,$0.000000289829,26.2%,shib-original-vision,shib-original-vision,sov,0x2c5bc2ba3614fd27fcc7022ea71d9172e2632c16\nSmarDex,$0.005946941927,26.0%,smardex,smardex,sdex,0x5de8ab7e27f6e7a1fff3e5b337584aa43961beef\nUnizen,$0.103212,22.2%,unizen,unizen,zcx,0xc52c326331e9ce41f04484d3b5e5648158028804\nXEN Crypto,$0.000003467889,21.2%,xen-crypto,xen-crypto,xen,0x06450dee7fd2fb8e39061434babcfc05599a6fb8\n0x,$0.346002,17.9%,0x,0x,zrx,0xe41d2489571d322189246dafa5ebde1f4699f498\nSomnium Space CUBEs,$1.59,17.2%,somnium-

In [38]:
import pandas as pd
from zipfile import ZipFile
from io import BytesIO

data = {}
with ZipFile(BytesIO(r.content)) as f:
    for file in f.namelist():
        data[file.split("/")[-1].replace(".csv", "")] = pd.read_csv(f.open(file))


In [40]:
data["results"]


,wallet,num_tokens,num_txns
0,0x84e5c8518c248de590d5302fd7c32d2ae6b0123c,3,3
1,0x7cb5648450bee34bd2d5977c0f3dbac0def2ebf9,3,3
2,0xfe7dd4c21c9efed8dbfee47586228229e7c591b4,3,3
3,0x30c5312d9cf0d873994f000e72f1cbf561d0209c,3,3
4,0x7bfee91193d9df2ac0bfe90191d40f23c773c060,3,3
...,...,...,...
84,0xd0be1fded5d964619b92b3672c08c43305529be0,4,64
85,0xcc67717e45e9b2e33e899e70123e9305a1eb6a75,4,71
86,0x62d63efdfdc7804666725f4cc8cb45c1a41a0809,3,72
87,0x382ffce2287252f930e1c8dc9328dac5bf282ba1,13,107
